**Импорт библиотек**

In [129]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import json
import re

**Парсинг стартового списка и разбиение на категории**

In [147]:
%%time
# Пользователь вводит последнюю часть адреса сайта
tournament_id = input("Введите идентификатор турнира: ")

# Формируем полный URL
url = f'https://lichess.org/swiss/{tournament_id}'
response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, 'html.parser')

# Находим все скрипты на странице
scripts = soup.find_all('script')

pages_number = None

# Ищем скрипт, содержащий ключ "nbPlayers"
for script in scripts:
    if 'nbPlayers' in str(script):
        # Используем регулярное выражение для поиска значения ключа "nbPlayers"
        match = re.search(r'"nbPlayers":\s*(\d+)', str(script))
        if match:
            pages_number = match.group(1)
            break

if pages_number:
    # Преобразование переменной pages_number в число
    pages_number = int(pages_number)

    # Деление на 10 и округление в большую сторону
    pages_number = -(-pages_number // 10)

    # Прибавление 3 страницы на тот случай когда во время парсинка участники могут продолжать добавлятся
    pages_number += 3

    # Преобразование в целое число
    pages_number = int(pages_number)

    print("Количество страниц с участниками =", pages_number-3)
else:
    print("Ключ 'nbPlayers' не найден на странице.")

# Пользователь вводит количество категорий (не более 10)    
num_categories = 0
while num_categories <= 0 or num_categories > 10:
    num_categories = int(input("Введите количество категорий (не более 10): "))
    if num_categories <= 0 or num_categories > 10:
        print("Неверное количество категорий. Пожалуйста, введите число от 1 до 10.")

categories = {}

# Пользователь вводит значения категорий
print("Вводите границы категорий от меньшей к большей")
for i in range(num_categories):
    
    category_value = int(input(f"Введите значение для категории {i+1} (верхняя граница рейтинга, просто число без знаков <>=): "))
    categories[f'<{category_value}'] = category_value

# Функция для определения категории игрока
def categorize_rating(rating):
    for category, value in categories.items():
        if rating < value:
            return category
    return f'без категории'
    #return f'>={value}'
    
    
# Переходим к парсингу стартового списка участников    
print('Идет парсинг таблицы, ждите...')
# Создадим пустой список для хранения данных таблицы standing
all_data = []


# Цикл для перебора страниц от 1 до pages_number
for page_num in range(1, pages_number):  
    url = f'https://lichess.org/swiss/{tournament_id}?page={page_num}&playerInfo='
    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    script_tag = soup.find('script', text=re.compile('standing'))

# Достаем словарь со списком участников из кода страницы
    if script_tag:
        standing_data = re.search(r'"standing":\{.*?"players":\[(.*?)\]\}', script_tag.text, re.DOTALL)

        if standing_data:
            standing_json = f'{{"players":[{standing_data.group(1)}]}}'
            standing_dict = json.loads(standing_json)

            players_data = standing_dict['players']
            all_data.extend(players_data)

# Создадим DataFrame из общих данных
players = []
ratings = []
points = []
tieBreaks = []
ranks = []
sheetMins = []

# Заполняем списки инофрмацией из словаря
for player in all_data:
    players.append(player['user']['name'])
    ratings.append(player['rating'])
    points.append(player['points'])
    tieBreaks.append(player['tieBreak'])
    ranks.append(player['rank'])
    sheetMins.append(player['sheetMin'])

# создаем датафрейм и сохраняем в него списки с именами и рейтингами участников    
df = pd.DataFrame({
    'name': players,
    'start_rating': ratings
})

# Удаляем дубликаты, тк в процессе парсинга некоторые записи могут перейти с одной страницы на другую
df = df.drop_duplicates(keep='first')

# Выполняем функцию разбиения на категории
df['category'] = df['start_rating'].apply(categorize_rating)

# Добавление столбца "category" на основе столбца "rating" с использованием lambda-функции
#df['category'] = df['start_rating'].apply(lambda x: '<1300' if x < 1300 else ('<1600' if x < 1600 else ('<1900' if x < 1900 else '<2300')))

# Выводится стартовая таблица с категориями
df

Введите идентификатор турнира: Dh590H5r
Количество страниц с участниками = 6
Введите количество категорий (не более 10): 3
Вводите границы категорий от меньшей к большей
Введите значение для категории 1 (верхняя граница рейтинга, просто число без знаков <>=): 1300
Введите значение для категории 2 (верхняя граница рейтинга, просто число без знаков <>=): 1500
Введите значение для категории 3 (верхняя граница рейтинга, просто число без знаков <>=): 1700
Идет парсинг таблицы, ждите...
Wall time: 9.16 s


,name,start_rating,category
0,KotDaVinchi,2133,без категории
1,ilyander,2036,без категории
2,nocreativitytoo,2434,без категории
3,Blessingsphiri,1917,без категории
4,Reuf,2009,без категории
5,chessplayersahovski,2021,без категории
6,SVYATOSHA7,1879,без категории
7,bestttttt,1863,без категории
8,tumi62,2041,без категории
9,RimalSavi,1972,без категории


**Сохранение стартового списка в отдельный файл**

In [4]:
# Укажите путь для сохранения CSV файла
file_path = 'Lichess_start_list.csv'
# Сохранение DataFrame в CSV файл
df_final.to_csv(file_path, index=False)

**Скачивание финальной таблицы**

In [148]:
url = 'https://lichess.org/api/swiss/'+tournament_id+'/results?as=csv'
# Чтение CSV файла по ссылке
df_final = pd.read_csv(url)
df_final

,Rank,Title,Username,Rating,Points,Tie Break,Performance
0,1,NaN,KotDaVinchi,2133,9.0,50.50,2259.0000
1,2,NaN,ilyander,2036,9.0,47.50,2124.1667
2,3,NaN,nocreativitytoo,2434,9.0,46.50,2235.7273
3,4,NaN,Blessingsphiri,1917,8.0,36.00,2275.1250
4,5,NaN,Reuf,2009,8.0,34.50,2071.0908
5,6,NaN,chessplayersahovski,2021,6.5,24.50,1894.0000
6,7,NaN,SVYATOSHA7,1879,6.5,24.00,1926.3334
7,8,NaN,bestttttt,1863,6.5,23.00,2018.3000
8,9,NaN,tumi62,2041,6.0,32.00,2191.0000
9,10,NaN,RimalSavi,1972,6.0,28.50,1964.0000


**Мердж стартовой и финальной таблицы**

In [149]:
results = pd.merge(df[['name', 'start_rating', 'category']], df_final, left_on='name', right_on='Username', how='right')
results_no_duplicates = results.drop_duplicates(keep='first')
results_no_duplicates = results_no_duplicates.drop(columns = ['name'], axis=1)
results_sorted = results_no_duplicates.sort_values(by=['Points', 'Tie Break'], ascending=False)
results_sorted = results_sorted[['Username','start_rating','Rating','Points','Tie Break','category']]
results_sorted = results_sorted.fillna('late_reg')
results_sorted

,Username,start_rating,Rating,Points,Tie Break,category
0,KotDaVinchi,2133,2133,9.0,50.50,без категории
1,ilyander,2036,2036,9.0,47.50,без категории
2,nocreativitytoo,2434,2434,9.0,46.50,без категории
3,Blessingsphiri,1917,1917,8.0,36.00,без категории
4,Reuf,2009,2009,8.0,34.50,без категории
5,chessplayersahovski,2021,2021,6.5,24.50,без категории
6,SVYATOSHA7,1879,1879,6.5,24.00,без категории
7,bestttttt,1863,1863,6.5,23.00,без категории
8,tumi62,2041,2041,6.0,32.00,без категории
9,RimalSavi,1972,1972,6.0,28.50,без категории


**Вывод финальных таблиц c топ 10 участниками по категориям**

In [150]:
df_top_10 = results_sorted.groupby('category').apply(lambda x: x.nlargest(10, 'Points')).reset_index(drop=True)
# Добавление столбца "Rank" со значениями от 1 до 10
df_top_10['Rank'] = df_top_10.groupby('category')['Points'].rank(method='first', ascending=False)
df_top_10 = df_top_10[['Rank','Username','start_rating','Rating','Points','Tie Break','category']]
df_top_10 = df_top_10.rename(columns={'Rating': 'new_rating'})
# Получаем уникальные значения категории
unique_categories = df_top_10['category'].unique()

# Группировка данных по уникальным категориям и вывод каждой отдельной таблицы
for category in unique_categories:
    data = df_top_10[df_top_10['category'] == category].copy()
    # Преобразование Rank в целое число
    data['Rank'] = data['Rank'].astype(int)  
    data_reset_index = data.reset_index(drop=True)
    print(f"Category: {category}")
    print(data_reset_index.to_string(index=False))
    print("\n")

Category: <1300
 Rank      Username  start_rating  new_rating  Points  Tie Break category
    1      Leointer          1222        1222     3.0       6.00    <1300
    2  Ivan_molodec          1123        1123     1.5       4.00    <1300
    3 abolfazle1392          1125        1125     1.5       0.00    <1300
    4   Stepan_2105          1285        1285     0.5       0.25    <1300
    5      chesshim          1133        1133     0.0       0.00    <1300
    6     akundapur           958         958     0.0       0.00    <1300


Category: <1500
 Rank            Username  start_rating  new_rating  Points  Tie Break category
    1              lara60          1402        1402     2.5        3.5    <1500
    2     mohsenmoosavi64          1354        1354     2.0        3.0    <1500
    3             VLAD_72          1385        1385     1.5        2.5    <1500
    4     KnightRider1994          1480        1480     1.5        1.5    <1500
    5 Fajer-shataranj6788          1394        1